In [32]:
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.datasets import Caltech101
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import random

# Phase 1:

## Data Import:

In [ ]:
# Local Datapath
dataset_path = "./caltech101"

# Transforms for normalization,  turning from PIL to tensor, resizing, transforming to RGB, and cropping
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    # Convert all images to RGB format before converting to a tensor
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224,    for image, label in iter(caltech_dataloader):
        target_label = random.choice(class_names)

        # Perform the attack
        # Run the attack
        adversarial_image, success = jsma_attack(
            model=model,
            original_image=image,
            original_label=label,
            target_label=target_label,
            theta=1.0, # Perturbation strength
            epsilon=0.2,  # Max percentage of pixels to change
            mask_labels=labels_to_remove
        )
        if success:
            print(f"Success!!")
        else:
            pass 0.225])
])

# Create dataset object
caltech_dataset =Caltech101(
    root=dataset_path,
    download=False,
    transform=transform
)

# Create dataloader
caltech_dataloader = DataLoader(caltech_dataset, batch_size=64, shuffle=True)

In [ ]:
# Splitting dataset into training and validation sets
train_size = int(0.8 * len(caltech_dataset))
val_size = len(caltech_dataset) - train_size
train_dataset, val_dataset = random_split(caltech_dataset, [train_size, val_size])

# Load Dataset using DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

## Performing the Transfer learning:

### Load pre-trained models:

#### Resnet-34

In [ ]:
resnet34 = models.resnet34(pretrained=True)

# Freeze all layers except the final layer
for param in resnet34.parameters():
    param.requires_grad = False

# Modify the final layer for 101 classes from FC to linear
num_ftrs = resnet34.fc.in_features
resnet34.fc = nn.Linear(num_ftrs, 101)

#### MobileNetV2

In [ ]:
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

# Freeze all layers except the final layer
for param in mobilenet_v2.parameters():
    param.requires_grad = False

# Modify the final layer for 101 classes from sequential to linear
num_ftrs = mobilenet_v2.classifier[1].in_features
mobilenet_v2.classifier[1] = nn.Linear(num_ftrs, 101)

### Train the last layer:

#### ResNet-34

In [ ]:
# Number of epochs
num_epochs = 5

# Initialize Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34.parameters(), lr=0.001)

# Train the last layer
for epoch in range(num_epochs):
    # Training
    resnet34.train()
    running_train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_loader)

    # Validation
    resnet34.eval()
    running_val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = resnet34(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = running_val_loss / len(val_loader)
    accuracy = correct_predictions / total_samples

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, "
          f"Validation Accuracy: {accuracy:.4f}")

print("Done")

#### MobileNet V2

In [ ]:
# Initialize Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet_v2.classifier[1].parameters(), lr=0.001)


# Train the last layer
for epoch in range(num_epochs):
    # Training
    mobilenet_v2.train()
    running_train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = mobilenet_v2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_loader)

    # Validation
    mobilenet_v2.eval()
    running_val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = mobilenet_v2(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = running_val_loss / len(val_loader)
    accuracy = correct_predictions / total_samples

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, "
          f"Validation Accuracy: {accuracy:.4f}")

print("Done")

# Phase 2:

## The JSMA Attack function:

In [35]:
def jsma_attack(model, original_image, original_label, mask_labels, target_label, theta=-0.1, epsilon=0.1):
    """
    Implements the Jacobian-based Saliency Map Attack (JSMA).
    
    Args:
        model (nn.Module): The neural network model to attack.
        original_image: The original input image.
        original_label: The true label of the image.
        target_label: The desired adversarial target class.
        theta: The perturbation amount added to each pixel.
        gamma: A scalar between 0 and 1 that controls the maximum
                       number of pixels to modify.
    
    Returns:
        torch.Tensor: The perturbed, adversarial image.
        bool: True if the attack was successful, False otherwise.
    """
    # Set the model to evaluation mode
    model.eval()
    
    # Clone the original image and enable gradient tracking
    adversarial_image = original_image.clone().detach()
    adversarial_image.requires_grad = True
    
    # Max perturbations allowed based on gamma
    max_perturbations = int(np.prod(original_image.shape) * epsilon)
    
    # Keep track of modified pixels to prevent redundant changes
    modified_pixels = set()
    
    # Determine the number of classes for the Jacobian calculation
    output = model(adversarial_image)
    num_classes = output.shape[1]

    # Check if label needs to be masked
    if original_label in mask_labels:
        return adversarial_image.detach(), True
    
    # The attack is an iterative process
    for _ in tqdm(range(max_perturbations), desc="JSMA Attack Progress"):
        # Forward pass to get the output logits
        output = model(adversarial_image)
        
        # Check if the attack has succeeded
        if output.argmax(dim=1).item() == target_label:
            print("Attack successful! Model classified as target label.")
            return adversarial_image.detach(), True

        # Compute the Jacobian Matrix
        # Initialize the Jacobian tensor with zeros
        jacobian = torch.zeros(num_classes, np.prod(adversarial_image.shape))
        
        # For each class, compute the gradient of its logit with respect to the input image
        for c in range(num_classes):
            # Zero out previous gradients
            if adversarial_image.grad is not None:
                adversarial_image.grad.zero_()
            
            # Compute the gradient of the current class's output
            output[0, c].backward(retain_graph=True)
            
            # Flatten the gradient and store it in the Jacobian matrix
            jacobian[c] = adversarial_image.grad.view(-1).clone()

        # Construct the Saliency Map
        # Get the Jacobian for the target class and for all other classes
        target_jacobian = jacobian[target_label]
        other_jacobians = jacobian[np.arange(num_classes) != target_label].sum(dim=0)
        
        # Saliency map calculation based on the paper's formula
        saliency_map = target_jacobian * (other_jacobians + target_jacobian)
        
        # Mask out pixels that have already been modified
        for pixel_idx in modified_pixels:
            saliency_map[pixel_idx] = -1 # A negative value to ensure it's not chosen
        
        # Find the pixel with the highest saliency score
        pixel_to_change = torch.argmax(saliency_map)
        
        # If no valid pixel can be found, stop the attack
        if saliency_map[pixel_to_change] <= 0:
            print("No suitable pixels found. Attack failed.")
            return adversarial_image.detach(), False
        
        # Modify the selected pixel
        # Add the perturbation to the selected pixel
        adversarial_image.data.view(-1)[pixel_to_change] += theta
        
        # Clamp the pixel value to be within the valid range [0, 1]
        adversarial_image.data = torch.clamp(adversarial_image.data, 0, 1)
        
        # Add the modified pixel to the set
        modified_pixels.add(pixel_to_change.item())
        
        # Check if the adversarial image is still valid
        if torch.equal(original_image, adversarial_image):
            print("No change applied. Attack failed.")
            return adversarial_image.detach(), False
            
    # Attack failed if the loop completes without success
    print("Attack failed. Maximum perturbations reached.")
    return adversarial_image.detach(), False


## Perform the attack:

In [ ]:
for c in caltech_dataset.categories:
    if c == "Faces_easy":
        print(c)

Faces


In [ ]:
# Set the model
model = resnet34

print("Performing a JSMA attack on the dataset...\n")

# Remove human and car like
labels_to_remove = ["Faces", "Faces_easy", "Motorbikes", "car_side"]
class_names = caltech_dataset.categories
for c in labels_to_remove:
    class_names.remove(c)

# Perform the attack on each image in the dataloader
try:
    for image, label in iter(caltech_dataloader):
        target_label = random.choice(class_names)

        # Perform the attack
        # Run the attack
        adversarial_image, success = jsma_attack(
            model=model,
            original_image=image,
            original_label=label,
            target_label=target_label,
            theta=1.0, # Perturbation strength
            epsilon=0.2,  # Max percentage of pixels to change
            mask_labels=labels_to_remove
        )
        if success:
            print(f"Success!!")
        else:
            pass
except:
    pass

print("Done")

Performing a JSMA attack on the dataset...


Done


# Phase 3

# Phase 4:

# Phase 5: